In [ ]:
from alpaca_trade_api.rest import REST, TimeFrame
import pandas as pd


In [ ]:
BASE_URL = "https://paper-api.alpaca.markets"
KEY_ID = "PKFWIFTNWGL03QMC6IY6"
SECRET_KEY = "f8EuHK0EIDbdK3xqnmd3kEq1JARwScJF2tkJhafk"

In [ ]:
# Instantiate REST API Connection
api = REST(key_id=KEY_ID,secret_key=SECRET_KEY,base_url="https://paper-api.alpaca.markets")

# Fetch 1Minute historical bars of Bitcoin
bars = api.get_crypto_bars("BTCUSD", TimeFrame.Minute).df
print(bars)

In [ ]:
# Filter data by exchange
bars = bars[bars.exchange == 'CBSE']
print(bars)

In [ ]:
 # Create a market order to buy 1 Bitcoin
order_buy = api.submit_order('BTCUSD', qty=1, side='buy')
print(order_buy)

In [ ]:
#create a market order to sell 1 bitcoin
order_sell = api.submit_order('BTCUSD',qty=1, side='sell')
print(order_sell)

In [ ]:
# get current position on bitcoin. yields error if we dont hold any
api.get_position('BTC')
#get all position and find out if we hold any bitcoin
positions = api.list_positions()
position_qty = 0
for p in positions:
    if p.symbol == 'BTCUSD':
        position_qty = float(p.qty)
print(position_qty) 

In [ ]:
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0

In [ ]:
import random

SYMBOL = 'BTCUSD'
while True:
    # GET OUR CURRENT POSITION
    position = get_position(symbol=SYMBOL)
    
    # SCIENTIFICALLY CHECK IF WE SHOULD BUY OR SELL
    gods_say_buy = random.choice([True, False])
    print(f"Holding: {position} / Gods: {gods_say_buy}")

    #CHECK IF WE SHOULD BUY
    if position == 0 and gods_say_buy == True:
        # WE BUY ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='buy')
    #HECK IF WE SHOULD SELL
    elif position > 0 and gods_say_buy == False:
        # WE SELL ONE BITCOIN
        print('The gods have spoken:')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: 1')
        api.submit_order(SYMBOL, qty=1, side='sell')
    print('Lets wait for the gods to manifest again...')
    print("*"*20)
    time.sleep(10)

In [ ]:
from datetime import datetime, timedelta
import math
import time

SYMBOL = 'BTCUSD'
SMA_FAST = 12
SMA_SLOW = 24
QTY_PER_TRADE = 1


# Description is given in the article
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

# Same as the function in the random version
def get_position(symbol):
    positions = api.list_positions()
    for p in positions:
        if p.symbol == symbol:
            return float(p.qty)
    return 0


# Returns a series with the moving average
def get_sma(series, periods):
    return series.rolling(periods).mean()

# Checks wether we should buy (fast ma > slow ma)
def get_signal(fast, slow):
    print(f"Fast {fast[-1]}  /  Slow: {slow[-1]}")
    return fast[-1] > slow[-1]

# Get up-to-date 1 minute data from Alpaca and add the moving averages
def get_bars(symbol):
    bars = api.get_crypto_bars(symbol, TimeFrame.Minute).df
    bars = bars[bars.exchange == 'CBSE']
    bars[f'sma_fast'] = get_sma(bars.close, SMA_FAST)
    bars[f'sma_slow'] = get_sma(bars.close, SMA_SLOW)
    return bars

while True:
    # GET DATA
    bars = get_bars(symbol=SYMBOL)
    # CHECK POSITIONS
    position = get_position(symbol=SYMBOL)
    should_buy = get_signal(bars.sma_fast,bars.sma_slow)
    print(f"Position: {position} / Should Buy: {should_buy}")
    if position == 0 and should_buy == True:
        # WE BUY ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='buy')
        print(f'Symbol: {SYMBOL} / Side: BUY / Quantity: {QTY_PER_TRADE}')
    elif position > 0 and should_buy == False:
        # WE SELL ONE BITCOIN
        api.submit_order(SYMBOL, qty=QTY_PER_TRADE, side='sell')
        print(f'Symbol: {SYMBOL} / Side: SELL / Quantity: {QTY_PER_TRADE}')

    time.sleep(get_pause())
    print("*"*20)